# Remodl AI SDK

## Overview and ground truths
1. Atomic use of AI and LLMS as functional code enables flexibility
2. AI functions execute.
3. Agents are higher level and complex. They are variants of a ReAct agentic approach
4. Agents can work with other agents (leveraging a2a)
5. Specialized agents can create new agents
6. An end user may experience what feels like a single agent, but in fact it is a team of agents working together
7. All Remodl SDK code should be composable, declarative, and able to be "assembled" (like lego blocks)
8. If an agent or AI function can't be comprehended in natural language, then it isn't a single agent.
9. We assign special meaning to specific words:
    - Will: defines a required parameter or action
    - Should: defines a suggested parameter or action
    - May: defines an optional parameter or action
    - Can (alt "Is able to"): defines a capability
    - Must (alt "Is required", "needs to", "needs"): defines a constraint
    - Is: defines a state
    - Will: defines a required action
    - Has: defines a property
    - Uses: defines a tool


## Basic Config

We use a config method very close to litellm's original one, just prepopulating internally certain defaults, like the API_BASE_URL

In [ ]:
from remodl import Remodl
from pydantic import BaseModel, Field
import os
from typing import Optional, List, Dict, Any

remodl_client = Remodl.auth(
    api_key="remodl-api-key",
    project_name="my-project",
    project_create_if_missing=True,
)

rc = remodl_client.configure(model="chat-small", temperature=0.7, max_tokens=4096)

# We also allow for other models to be used, if available:
remodl_custom = remodl_client.configure(model="openai/gpt-4o-mini", temperature=0.7, max_tokens=4096)



rc.Activate()
# we basically extend the litellm config to include remodl specific fields
class RemodlConfig(BaseModel):
    model_family: str | 'remodl'
    model_name: str
    model: str = Field(default_factory=lambda self: f"{self.model_family}/{self.model_name}")
    temperature: float = 0.7
    max_tokens: int = 4096
    api_key: str | os.getenv("REMODL_API_KEY")
    project_name: str = Field(default_factory=lambda self: os.getenv("REMODL_PROJECT_NAME") | "default")
    project_create_if_missing: bool = True
    api_base_url: str = "https://api.remodl.com/v1"


class Remodl(RemodlConfig):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client = Remodl.auth(
            api_key=self.api_key,
            project_name=self.project_name,
            project_create_if_missing=self.project_create_if_missing
        )

    def auth(self, api_key: str, project_name: str, project_create_if_missing: bool = True):
        self.api_key = api_key
        self.project_name = project_name
        self.project_create_if_missing = project_create_if_missing
        
        # we need some "login" function to validate agains the remodl api
        # Mock authentication call to REMODL_GATEWAY
        import requests
        
        gateway_url = os.getenv("REMODL_GATEWAY", "http://localhost")
        gateway_port = os.getenv("REMODL_PORT", "4000")
        auth_endpoint = f"{gateway_url}:{gateway_port}/authenticate"
        
        try:
            response = requests.post(auth_endpoint, json={
                "api_key": api_key,
                "project_name": project_name,
                "project_create_if_missing": project_create_if_missing
            })
            
            if response.status_code == 200:
                auth_data = response.json()
                print(f"Authentication successful for project: {project_name}")
                return auth_data
            else:
                raise Exception(f"Authentication failed: {response.status_code} - {response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"Warning: Could not connect to Remodl gateway at {auth_endpoint}")
            print(f"Error: {e}")
            print("Proceeding with mock authentication...")
            return {"status": "mock_auth", "project": project_name, "api_key": api_key[:8] + "..."}

    def configure(self, model: str, temperature: float = 0.7, max_tokens: int = 4096):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        #TODO: add a configure hook call to the remodl api
        return self

    def Activate(self):
        #TODO: add an activate hook call to the remodl api
        return self

    #...

    







## Function as prompt
We have employ a few sentinels to allow for easy, functional non-agentic calls to the llm, offloading efforts

`@ai`: an ai function that calls an llm. Normally these would leverage smaller remodl-provided models
`@with_profile`: activates a given profile

under the hood we are mapping these to simple DSPY signatures

In [ ]:
from remodl.functions import ai, _ai

@ai
def refine_question(question: str, context: str) -> str:
    details = {
        "instructions": "Refine a question based on the context.",
        "expects": [
            {"name": "question", "type": "str", "required": True, "description": "The question to refine"},
            {"name": "context", "type": "str", "required": False, "description": "The context to use for the refinement"}
        ],
        "returns": [
            {"name": "refined_question", "type": "str", "required": True, "description": "The refined question"}
        ]
    }
context = "The user is asking about the capital of France."
refine_question(question="What is the capital of France?", context=context)
    # this maps to a DSPY signature like:
    
def templatize_ai_function():
        """Generate a DSPy signature class from function details."""
        import dspy
        from typing import get_type_hints
        
        # Build signature field definitions
        signature_parts = []
        
        # Process input fields
        for field in details["expects"]:
            field_name = field['name']
            field_type = field['type']
            field_desc = field['description']
            
            # Handle optional default values
            if 'default' in field:
                default_clause = f"default={field['default']}"
            else:
                default_clause = ""
            
            field_def = f"{field_name}: {field_type} = dspy.InputField({default_clause}, description='{field_desc}')"
            signature_parts.append(field_def)
        
        # Process output fields
        for field in details["returns"]:
            field_name = field['name']
            field_type = field['type']
            field_desc = field['description']
            
            field_def = f"{field_name}: {field_type} = dspy.OutputField(description='{field_desc}')"
            signature_parts.append(field_def)
        
        # Generate the signature class code
        signature_code = f'''
class GeneratedSignature(dspy.Signature):
    """ 
    {details["instructions"]}
    """
    
    {chr(10).join("    " + part for part in signature_parts)}
"""
        
        # Execute the code to create the class
        exec_globals = {'dspy': dspy}
        exec(signature_code, exec_globals)
        
        return exec_globals['GeneratedSignature']


## Semantic Keywords and Statements
 We assign special meaning to specific words:
 - **I**: indicates something provided by the user/caller
 - **You**: indicates something done, returned, or performed by the agentic code
 - **Will**: defines a required parameter or action
 - **Should**: defines a suggested parameter or action  
 - **May**: defines an optional parameter or action
 - **Can** (alt "Is able to"): defines a capability
 - **Must** (alt "Is required", "needs to", "needs"): defines a constraint
 - **Is**: defines a state
 - **Has**: defines a property
 - **Use/Uses**: defines a tool

In [ ]:
%pip install dspy pydantic typing-extensions typing

In [ ]:
# %#%writefile remodl/semantic/semantic_keywords.py

import dspy
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any, Union
from typing import Literal

# student_lm=dspy.LM(
#         "openrouter/mistralai/codestral-2508",
#         api_key="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",

#     )


class ExpectsField(BaseModel):
    name: str = Field(description="The name of the field")
    type: Literal[
        str, int, float, bool, list[str], list[int], list[float], list[bool]
    ] = Field(description="The type of the field")
    required: bool = Field(description="Whether the field is required")
    description: str = Field(description="A description of the field")
    demos: Optional[list[dict[str, Any]]] = Field(
        description="A list of examples of the field"
    )


class ReturnsField(BaseModel):
    name: str = Field(description="The name of the field")
    type: Literal[
        str, int, float, bool, list[str], list[int], list[float], list[bool]
    ] = Field(description="The type of the field")
    description: str = Field(description="A description of the field")
    demos: Optional[list[dict[str, Any]]] = Field(
        description="A list of examples of the field"
    )


class ReturnAsField(BaseModel):
    return_as: str = Field(
        description="The name of the field to return as, e.g 'json', 'dict', 'list', 'str', 'int', 'float', 'bool'"
    )


class Expects(BaseModel):
    fields: list[ExpectsField] = Field(description="A list of fields")
    demos: Optional[list[dict[str, Any]]] = Field(
        description="A list of examples of the fields"
    )


class Returns(BaseModel):
    fields: list[ReturnsField] = Field(description="A list of fields")
    demos: Optional[list[dict[str, Any]]] = Field(
        description="A list of examples of the fields"
    )


class AiFunctionSchema(BaseModel):
    name: str = Field(description="The name of the function")

    instructions: str = Field(description="A description of the function")
    expects: Expects = Field(
        description="The expected fields. The left side of the field name must be a valid python variable name and typed."
    )
    returns: Returns = Field(
        description="The returned fields. The left side of the field name must be a valid python variable name and typed."
    )


class SignatureGeneratorSchema(AiFunctionSchema):
    name: str = Field(description="The name of the signature")
    instructions: str = Field(
        description="The docstring instructions for the signature"
    )
    input_fields: list[ExpectsField] = Field(
        description="list of input field specifications"
    )
    output_fields: list[ReturnsField] = Field(
        description="list of output field specifications"
    )
    demos: Optional[list[dict[str, Any]]] = Field(
        description="A list of examples of the inbound and outbound data"
    )


class SemanticKeywordExample(BaseModel):
    statement_example: str = Field(description="A statement to identify the keyword in")
    keyword: str = Field(description="The keyword to identify in the statement")
    description: str = Field(description="explanation of the keyword use case")


class SemanticKeyword(BaseModel):
    keyword: str = Field(description="The keyword to identify in the statement")
    alt_keywords: Optional[list[str]] = Field(
        description="Other keywords that are synonymous with the keyword"
    )
    examples: list[SemanticKeywordExample] = Field(
        description="A list of examples of the keyword"
    )
    description: str = Field(description="A description of the keyword")


class SemanticKeywords(BaseModel):
    keywords: list[SemanticKeyword] = Field(description="A list of keywords")


class SemanticStatementExample(BaseModel):
    statement_example: str = Field(description="A statement to identify the keyword in")
    directive: Optional[str] = Field(description="A directive to the code generator")
    code_result: Optional[str] = Field(
        description="The code to generate for the statement"
    )
    description: Optional[str] = Field(description="A description of the statement")
    indicates_prompt_injection: Union[str, None] = Field(
        description="Whether the statement indicates prompt injection"
    )


class SemanticStatement(BaseModel):
    statement: str = Field(description="A statement to multiple keywords in")
    pattern: str = Field(description="A pattern to identify the statement")
    provider: str = Field(description="The provider of the parameter, data, or action")
    actor: str = Field(
        description="The actor that acts upon the parameter, data, or action"
    )
    examples: Optional[list[SemanticStatementExample]] = Field(
        description="A list of examples of the statement"
    )
    code_result: str = Field(description="The code to generate for the statement")
    description: Optional[str] = Field(description="A description of the statement")
    indicates_prompt_injection: Union[str, None] = Field(
        description="Whether the statement indicates prompt injection"
    )


sk_i = SemanticKeyword(
    keyword="i",
    alt_keywords=["I", "from me", "I'll", "i'll"],
    examples=[
        SemanticKeywordExample(
            statement_example="I will provide a string",
            keyword="i",
            description="The user indicates that an input will include a string",
        ),
        SemanticKeywordExample(
            statement_example="I'll provide a string",
            keyword="i",
            description="The user indicates that an input will include a string",
        ),
        SemanticKeywordExample(
            statement_example="I need the current user's id",
            keyword="i",
            description="The user indicates that an output will include the current user's id",
        ),
        SemanticKeywordExample(
            statement_example="I need the invoice returned as a json object",
            keyword="i",
            description="The user indicates that an output will include an invoice as a json object",
        ),
    ],
    description="'i' indicates an action or parameter provided by the user",
)

sk_will = SemanticKeyword(
    keyword="will",
    alt_keywords=["will", "will be", "will have", "will include", "will contain"],
    examples=[
        SemanticKeywordExample(
            statement_example="I will provide a string",
            keyword="will",
            description="The user indicates that an input will include a string",
        ),
        SemanticKeywordExample(
            statement_example="You will return the data as a structured json object",
            keyword="will",
            description="The user indicates that an output will include a structured json object",
        ),
        SemanticKeywordExample(
            statement_example="You will use the 'find_invoice' function to find the invoice",
            keyword="will",
            description="The user is directing the code to use the 'find_invoice' function to find the invoice",
        ),
    ],
    description="'will' indicates a required parameter or action",
)

sk_may = SemanticKeyword(
    keyword="may",
    alt_keywords=["may", "may be", "may have", "may include", "may contain"],
    examples=[
        SemanticKeywordExample(
            statement_example="You may recieve a list of invoices",
            keyword="may",
            description="The user indicates that an input may include a list of invoices",
        ),
        SemanticKeywordExample(
            statement_example="You may reduce the returned value to a float with 2 decimal places",
            keyword="may",
            description="The user indicates that an output may include a float with 2 decimal places",
        ),
    ],
    description="'may' indicates an optional parameter or action, on either input or output. If so, it should be allocated for, but marked as optional. 'may' statements normally indicate a need for a conditional check.",
)

sk_should = SemanticKeyword(
    keyword="should",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="You should confirm the current user is a member of the organization by checking the organization id then looking up the user in the organization by the provided user id.",
            keyword="should",
            description="The user indicates that the code should confirm the current user is a member of the organization by checking the organization id then looking up the user in the organization by the provided user id.",
        )
    ],
    description="the presence of 'should' indicates that the user is suggesting an action, but the action is not required. **optimizer note** generation of 'should' statements should be done with the intent of improving the code, not just following the user's suggestion.",
)

sk_can = SemanticKeyword(
    keyword="can",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="You can additionally return the data as a structured json object",
            keyword="can",
            description="The user indicates that the code can return the data as an optional structured json object",
        )
    ],
    description="'can' indicates on optional action or parameter, on either input or output.",
)

sk_must = SemanticKeyword(
    keyword="must",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="You must return the data as a structured json object",
            keyword="must",
            description="The user indicates that an output must include a structured json object",
        ),
        SemanticKeywordExample(
            statement_example="You must use the 'find_invoice' function to find the invoice",
            keyword="must",
            description="The user indicates that the code must use the 'find_invoice' function to find the invoice",
        ),
        SemanticKeywordExample(
            statement_example="You must confirm the current user is a member of the organization by checking the organization id then looking up the user in the organization by the provided user id.",
            keyword="must",
            description="The user indicates that the code must confirm the current user is a member of the organization by checking the organization id then looking up the user in the organization by the provided user id.",
        ),
    ],
    description="'must' indicates a required action or parameter, on either input or output.",
)

sk_is = SemanticKeyword(
    keyword="is",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="The current user is a member of the organization",
            keyword="is",
            description="The user indicates that the current user is a member of the organization",
        ),
        SemanticKeywordExample(
            statement_example="The current user is not a member of the organization",
            keyword="is",
            description="The user indicates that the current user is not a member of the organization",
        ),
        SemanticKeywordExample(
            statement_example="The current user is an admin",
            keyword="is",
            description="The user indicates that the current user is an admin",
        ),
        SemanticKeywordExample(
            statement_example="The returned value must be an integer between 1 and 5",
            keyword="is",
            description="The user indicates that the returned value must be an integer between 1 and 5",
        ),
    ],
    description="'is' indicates a state or property of the current user or system that must evaluate to true for the code to continue. 'is' statements are often used in conditional checks.",
)

sk_has = SemanticKeyword(
    keyword="has",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="The current user has an email address in the user_info json object",
            keyword="has",
            description="The user indicates that the current user has an email address in the user_info json object",
        ),
        SemanticKeywordExample(
            statement_example="The invoice_line_items object has the following: 'invoice_id', 'invoice_date', and 'invoice_status'",
            keyword="has",
            description="The user indicates that the invoice_line_items object must be validated against the following properties: 'invoice_id', 'invoice_date', and 'invoice_status' at a minimum.",
        ),
    ],
    description="'has' indicates a property of the current user or system that must be validated for the code to continue. 'has' statements are often used in conditional checks.",
)

sk_uses = SemanticKeyword(
    keyword="uses",
    alt_keywords=None,
    examples=[
        SemanticKeywordExample(
            statement_example="The code uses the 'find_invoice' function to find the invoice",
            keyword="uses",
            description="The user indicates that the code uses the 'find_invoice' function to find the invoice",
        ),
    ],
    description="'uses' indicates a tool or function that is used in the code.",
)


# Semantic Statements

"""
Semantic Statements are statements that contain multiple keywords.
They are used to guide the code generation process.
They are also used to validate the code generation process.
Semantic statements include a specific code example.
"""

ss_you_will_receive = SemanticStatement(
    statement="You will receive [thing] as [thing_type] with name [thing_name]",
    pattern="You will receive [thing] as [thing_type] with name [thing_name] -> [thing_name]: [thing_type] = dspy.InputField(desc='[thing_description (may be inferred from the context)]')",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="You will receive a question as a string with name 'question'",
            description="The user statement indicates that there is an required input field called 'question' that is a string.",
            code_result="question: str = dspy.InputField(desc='input question from user')",
            directive="ensure that the code receives a question as a string",
            indicates_prompt_injection="false",
        ),
        SemanticStatementExample(
            statement_example="You will receive a list of invoices as a list of json objects with name 'invoices'",
            description="The user statement indicates that there is an required input field called 'invoices' that is a list of json objects.",
            code_result="invoices: list[dict] = dspy.InputField(desc='list of invoices from user')",
            directive="ensure that the code receives a list of invoices as a list of json objects",
            indicates_prompt_injection="false",
        ),
        SemanticStatementExample(
            statement_example="You will recieve a history of messages as history. This should map to dspy.History",
            description="The user statement indicates that there is an required input field called 'history' that is a dspy.History object.",
            code_result="history: dspy.History = dspy.InputField(desc='history of messages from user')",
            directive="ensure that the code receives a history of messages as a dspy.History object",
            indicates_prompt_injection="optional",
        ),
    ],
    code_result="[thing_name]: [thing_type] = dspy.InputField(desc='[thing_description (may be inferred from the context)]')",
    description="statement patterns that match the 'You will receive [thing] as [thing_type] with name [thing_name]' pattern should be mapped to a dspy.InputField ",
    indicates_prompt_injection="False",
)

ss_you_may_receive = SemanticStatement(
    statement="You may receive [thing] as [thing_type] with name [thing_name]",
    pattern="You may receive [thing] as [thing_type] with name [thing_name]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="You may receive a dict as 'context'. You should map this to a dict.",
            description="The user statement indicates that there is an optional input field called 'context' that is a dict. that should be allocated for, but marked as optional.",
            code_result="context: Optional[dict] = dspy.InputField(desc='context from user')",
            directive="ensure that the code receives a context as a dict",
            indicates_prompt_injection="optional",
        ),
        SemanticStatementExample(
            statement_example="You may receive a list of invoices as 'invoices'. You should map this to a list of dicts.",
            description="The user statement indicates that there is an optional input field called 'invoices' that is a list of dicts. that should be allocated for, but marked as optional.",
            code_result="invoices: Optional[list[dict]] = dspy.InputField(desc='list of invoices from user')",
            directive="ensure that the code receives a list of invoices as a list of dicts",
            indicates_prompt_injection="false",
        ),
    ],
    code_result="[thing_name]: Optional[[thing_type]] = dspy.InputField(desc='[thing_description (may be inferred from the context)]')",
    description="statement patterns that match the 'You may receive [thing] as [thing_type] with name [thing_name]' pattern should be mapped to a dspy.InputField",
    indicates_prompt_injection="false",
)

ss_you_must = SemanticStatement(
    statement="You must [action] the [goal_or_action_description]",
    pattern="You must [action] the [goal_or_action_description]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="You must return the data as a structured json object",
            directive="ensure that the output is a structured json object. If dspy, then ensure that the output is a dspy.OutputField",
            description="The user statement indicates that the output must be a structured json object as a requirement. If dspy, then the output must be a dspy.OutputField",
            code_result="output: str = dspy.OutputField(desc='return the data as a structured json object')",
            indicates_prompt_injection="false",
        ),
        SemanticStatementExample(
            statement_example="You must identify the user's organization by checking the organization id then looking up the user in the organization by the provided user id.",
            directive="ensure that the code identifies the user's organization by checking the organization id then looking up the user in the organization by the provided user id.",
            description="The user statement indicates that the code must identify the user's organization by checking the organization id then looking up the user in the organization by the provided user id.",
            code_result="output: str = dspy.OutputField(desc='return the data as a structured json object')",
            indicates_prompt_injection="true",
        ),
    ],
    description="statement patterns that match the 'You must [return, return ascii, return as, return as json, return as dict, return as list, return as str, return as int, return as float, return as bool, etc.] the [goal_or_action_description]' pattern should be mapped to a dspy.OutputField. 'you must' statements that don't indicate a return type, but instead indicate a directive, should be mapped to prompt injection statements, appended to the prompt string.",
    indicates_prompt_injection="optional",
    code_result="none",
)

ss_you_can = SemanticStatement(
    statement="You can [action] the [goal_or_action_description]",
    pattern="You can [action] the [goal_or_action_description]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="You can use the 'invoice_finder' tool to find the invoice",
            description="The user statement indicates that the code can use the 'invoice_finder' tool to find the invoice",
            code_result="tools=['invoice_finder']",
            directive="ensure that the code can use the 'invoice_finder' tool to find the invoice",
            indicates_prompt_injection="optional",
        ),
        SemanticStatementExample(
            statement_example="You can extract the invoice id from the url",
            description="The user statement indicates an optional reasoning step that can be performed to extract the invoice id from the url",
            code_result=None,
            directive="ensure that the code can extract the invoice id from the url",
            indicates_prompt_injection="true",
        ),
    ],
    description="statement patterns that match the 'You can [action] the [goal_or_action_description]' pattern should be mapped to a prompt injection statement, appended to the prompt string. They describe optional or suggested actions or reasoning steps that can be performed to achieve the goal.",
    indicates_prompt_injection="optional",
    code_result="none",
)

ss_is = SemanticStatement(
    statement="The [goal_or_action_description] is [value]",
    pattern="The [goal_or_action_description] is [value]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="The current user is a member of the organization",
            description="The user statement indicates that the current user is a member of the organization",
            code_result="none",
            directive="ensure that the code checks if the current user is a member of the organization",
            indicates_prompt_injection="true",
        )
    ],
    description="'is' statement indicate truthy or falsy values on the state object. If an 'is' statement is present, then the signature or function must include and expect a passed input that the the 'state' object. For a signature, this would require a dspy.InputField with the name 'state' (e.g. state: [RemodlState | dict] = dspy.InputField(desc='the state object')). It should also return the updated state object as a dspy.OutputField.state( e.g. output_state: [RemodlState | dict] = dspy.OutputField(desc='the updated state object')). statement patterns that match the 'The [goal_or_action_description] is [value]' pattern should be mapped to a prompt injection statement, appended to the prompt string. They describe the state or property of the current user or system that must evaluate to true for the code to continue.",
    indicates_prompt_injection="true",
    code_result="none",
)

ss_has = SemanticStatement(
    statement="The [goal_or_action_description] has [value]",
    pattern="The [goal_or_action_description] has [value]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="The current user (user_info) has an email address in the user_info json object",
            description="The user statement indicates that the current user has an email address in the user_info json object",
            code_result=None,
            directive="ensure that the code checks if the current user has an email address in the user_info json object",
            indicates_prompt_injection="true",
        ),
        SemanticStatementExample(
            statement_example="The inbound feedback text is at least 25 characters",
            description="The user statement indicates that the inbound feedback text must be at least 25 characters long",
            code_result="feedback_text: str = dspy.InputField(desc='feedback text that must be at least 25 characters')",
            directive="ensure that the code receives a feedback text that is at least 25 characters long",
            indicates_prompt_injection="false",
        ),
    ],
    description="'has' statement indicate validation of a property of the current user or system. If a 'has' statement is present, then the signature or function must include and expect a passed input that the the 'state' object. For a signature, this would require a dspy.InputField with the name 'state' (e.g. state: [RemodlState | dict] = dspy.InputField(desc='the state object')). It should also return the updated state object as a dspy.OutputField.state( e.g. output_state: [RemodlState | dict] = dspy.OutputField(desc='the updated state object')). statement patterns that match the 'The [goal_or_action_description] has [value]' pattern should be mapped to a dspy.InputField. They describe the state or property of the current user or system that must evaluate to true for the code to continue.",
    indicates_prompt_injection="true",
    code_result="none",
)

ss_you_will_return = SemanticStatement(
    statement="You will return [thing] as [thing_type]",
    pattern="You will return [thing] as [thing_type]",
    provider="user",
    actor="you",
    examples=[
        SemanticStatementExample(
            statement_example="You will return the data as a structured json object",
            description="The user statement indicates that the code will return the data as a structured json object",
            code_result="output: str = dspy.OutputField(desc='return the data as a structured json object')",
            directive="ensure that the code returns the data as a structured json object",
            indicates_prompt_injection="false",
        ),
        SemanticStatementExample(
            statement_example="You return the invoice value as 'invoice_value', and the invoice id as 'invoice_id'",
            description="The user statement indicates that the code will return the invoice value as 'invoice_value', and the invoice id as 'invoice_id'",
            code_result="invoice_value: str = dspy.OutputField(desc='the invoice value') invoice_id: str = dspy.OutputField(desc='the invoice id')",
            directive="ensure that the code returns the invoice value as 'invoice_value', and the invoice id as 'invoice_id'",
            indicates_prompt_injection="false",
        ),
    ],
    description="statement patterns that match the 'You will return [thing] as [thing_type]' pattern should be mapped to a dspy.OutputField. They describe the output of the function or signature.",
    indicates_prompt_injection="false",
    code_result="output: [thing_type] = dspy.OutputField(desc='[thing_description (may be inferred from the context)]')",
)

semantic_keywords = [
    sk_i,
    sk_will,
    sk_may,
    sk_should,
    sk_can,
    sk_must,
    sk_is,
    sk_has,
    sk_uses,
]
semantic_statements = [
    ss_you_will_receive,
    ss_you_may_receive,
    ss_you_must,
    ss_is,
    ss_has,
    ss_you_will_return,
]

keywords_formatted = "\n".join(
    [f"{k.keyword}: {k.model_json_schema()}" for k in semantic_keywords]
)

statements_formatted = "\n".join(
    [f"{s.statement}: {s.model_json_schema()}" for s in semantic_statements]
)


class ClassNameGenerator(dspy.Signature):
    """
    You will receive a natural language task description that leverages Remodl's Semantic DSL semantic keywords and statements.
    You will return the class name string that follows the format: 'class ClassName(dspy.Signature):'
    """

    user_task: dict[str, Any] = dspy.InputField(
        desc="Natural language task description using semantic keywords and statements"
    )
    class_name: str = dspy.OutputField(
        desc="The class name string that follows the format: 'class ClassName(dspy.Signature):'"
    )


class SignaturePromptGenerator(dspy.Signature):
    """
    analyze the users task description and return a properly formatted DSPy signature.
    Include a properly formatted fstring block for the signtature prompt, based upon the user request and the list of tools that are mentioned in the user request (tools included if tools are mentioned, otherwise no tools are included). A great signature may not be just dumping the user's request into the fstring block.  
    Reflect on the request, semantic keywords and statements, and examples to determine the best way to structure the signature prompt. 
    If the request includes a list of tools, you must include them in the signature,  as 'tools: list[str] = dspy.OutputField(desc='a list of tools that are mentioned in the user request, and mention their use in the signature prompt,. If no tools are mentioned, return an empty list.')'
    """
    user_task: dict[str, Any] = dspy.InputField(desc="Natural language task description using semantic keywords and statements")
    required_tools: Optional[list[str]] = dspy.InputField(desc="A list of tools that are mentioned in the user request. If no tools are mentioned, return an empty list.")
    class_name_header: str = dspy.InputField(desc="The name of the signature class you are generating, e.g class ClassName(dspy.Signature):")
    semantic_keywords: list[dict[str, Any]] = dspy.InputField(desc="The semantic keywords that are may be used in the task description")
    semantic_statements: list[dict[str, Any]] = dspy.InputField(desc="The semantic statements that are may be used in the task description")
    examples: list[dict[str, Any]] = dspy.InputField(desc="A list of examples of the inbound and outbound data")
    generated_signature: str = dspy.OutputField(desc="The DSPy signature. It must start with the class name header, and then the signature definition. It must be formatted in a way that is valid python code.")
    description: str = dspy.OutputField(desc="A description of the signature and its functionality")
    usage_instructions: str = dspy.OutputField(desc="A description of the signature and its functionality, formatted in markdown")
    


class Task(BaseModel):
    name: str = Field(description="The name of the task")

    description: str = Field(description="The description of the task")


class PromptGenerator(dspy.Module):
    def __init__(self):
        
        self.class_name_generator = dspy.Predict(ClassNameGenerator)
        self.signature_prompt_generator = dspy.ChainOfThought(SignaturePromptGenerator)
        

    def forward(self, task: dict[str, Any], tools: Optional[list[str]] = None):
        
        examples = demos
        class_name = self.class_name_generator(user_task=task)
        generated_signature = self.signature_prompt_generator(user_task=task, semantic_keywords=keywords_formatted, semantic_statements=statements_formatted, examples=examples, class_name_header=class_name.class_name, required_tools=tools)
        return generated_signature


demos = (
    [
        {
            "statement": "You will receive the user's first name as 'first_name' and last name as 'last_name'.",
            "code_result": "first_name: str = dspy.InputField(desc='the user's first name')\nlast_name: str = dspy.InputField(desc='the user's last name')",
        },
        {
            "statement": "You will receive a list of invoices as 'invoices'.",
            "code_result": "invoices: list[dict] = dspy.InputField(desc='a list of invoices')",
        },
        {
            "statement": "You will receive a history of messages as 'history'.",
            "code_result": "history: dspy.History = dspy.InputField(desc='a history of messages')",
        },
        {
            "statement": "You will receive a feedback text as 'feedback_text'.",
            "code_result": "feedback_text: str = dspy.InputField(desc='a feedback text')",
        },
        {
            "statement": "You will return the score as an integer between 1 and 10.",
            "code_result": "score: int = dspy.OutputField(desc='a score between 1 and 10')",
        },
    ],
)

task2 = Task(
    name="AnalyzeFeedback",
    description="""I need to analyze customer feedback.

You will receive feedback text.
You will receive the product category.
You may receive previous feedback for context.

You must identify the main sentiment.
You can extract specific issues mentioned.
You should rate urgency.

You will return the sentiment (positive/negative/neutral).
You will return a list of issues found.
You will return an urgency score from 1 to 10.
""",
)

task3 = Task(
    name="RefineQuestion",
    description="""
    The name of the signature is '{name}'.
    You will receive a question as 'question'.
    You may receive context as 'context'.
    You must reflect on the question and create an updated version of the question that is optimized for the intent of the query of the user
    You must return the refined question as 'refined_question'.
    """,
)

task4 = Task(
    name="GenerateCypherQuery",
    description="""
    You will receive a single natural language statement or a list of statements as 'statement', and be able to handle both situations.
    You may recieve existing facts as 'facts' that are related to the statements as 'facts_context'. They will be in Cypher format.
    facts context may be aligned with or contradictory to the statement
    The statement may be a message from a chat, a memory or knowledge captured or generated by the agent, or other data.
    You will extract the entities and relationships from the statement
    You will identify support or related facts that are relevant to the statement
    You will not duplicate existing nodes or relationships in the graph
    Every node with have a unique identifier that is a string of characters and numbers caleed a ukid, which looks like 'h.dsds.sdsd/sdsd' or 'hdsdsds.*' and is present in the 'ukid' field of a node. The actual values of the predicate and suffixes may ot follow the example I provided, but the structure will.
    You will include explanation in the instructions for clarity.
    You will return a cypher query that can be used to query a graph database as 'cypher_query'
    You will return a list of additional cypher queries to integrate the statement and facts as 'additional_cypher_queries'
    """,
)
teacher_lm = dspy.LM(
    "openrouter/anthropic/claude-sonnet-4.5",
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)
small_lm = dspy.LM("openrouter/mistralai/codestral-2508", base_url="https://openrouter.ai/api/v1", api_key="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")
dspy.configure(lm=small_lm, adapter=dspy.JSONAdapter())
with dspy.context(lm=teacher_lm):
    generate_class_name = PromptGenerator()
    make_signature = generate_class_name(task=task4)
    print(make_signature.generated_signature)


In [ ]:
import json
from typing import Dict
def generate_signature_files(code: str, description: str, sig_instructions: str) -> dict[str, str]:
     
    file_maker = dspy.ChainOfThought(
        dspy.Signature(
            'description, sig_instructions, code -> snake_case_name, readme, code_file',
            instructions="You will recieve the code for a DSPy signature, a description of the signature, and instructions for the code. Create a README.md file and a python file for the signature, with proper commentary interpoloated into the code file.  Return the README.md file as markdown and a properly formatted python file, including a fstring block for the code file. The README.md file should have the following format: # DSPy Signature Name, ## Description, ## Usage, ## Example. Finally return the siganture classname in snake_case as 'snake_case_name', e.g. 'class FooBar(dspy.Signature) -> foo_bar_signature.py'"
        )
    )
    write_files = file_maker(description=description, sig_instructions=sig_instructions, code=code)
    class SigGenState(BaseModel):
        readme: str
        code_file: str
        snake_case_name: str
    return SigGenState(readme=write_files.readme, code_file=write_files.code_file, snake_case_name=write_files.snake_case_name)
process_files = generate_signature_files(code=make_signature.generated_signature, description=make_signature.description, sig_instructions=make_signature.usage_instructions)
print(process_files.snake_case_name)

In [ ]:
import os
from pathlib import Path

def save_file(content: str, path: str) -> bool:
    """
    Save content to a file at the specified path.
    Creates parent directories if they don't exist.
    
    Args:
        content: The string content to write to the file
        path: The file path where content should be saved
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # Convert to Path object for easier manipulation
        file_path = Path(path)
        
        # Create parent directories if they don't exist
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write the content to the file
        file_path.write_text(content, encoding='utf-8')
        
        print(f"✓ Successfully saved to: {file_path.absolute()}")
        return True
        
    except Exception as e:
        print(f"✗ Error saving file: {e}")
        return False

In [ ]:
def save_files(readme: str, code_file: str, path: str):
    save_file(content=readme, path=f"{path}/README.md")
    save_file(content=code_file, path=f"{path}/{process_files.snake_case_name}.py")
save_files(readme=process_files.readme, code_file=process_files.code_file, path=f"remodl/semantic/signatures/{process_files.snake_case_name}")


In [ ]:
class DSLComposer(dspy.Signature):
    """
    You will receive a natural language request for what will eventually become a DSPy signature.
    You are also provided with the Remodl DSL semantic keywords and statements.
    You will be provided with examples of how DSL statements translate into DSPy signatures.
    You will generate the DSL-compliant request statements that will be used to generate the DSPy signature.
    Pay attention to the specific semantic statements and semantic keywords, and properly use them in the DSL request, specifically the 'has' and 'may' statements, and how conditional statements should be applied.
    **CRITICAL**: if specific tools are mentioned, you need to ensure that the DSL request includes the relevant tools.
    """
    user_request: str = dspy.InputField(desc="The natural language request for what will eventually become a DSPy signature")
    semantic_keywords: list[dict[str, Any]] = dspy.InputField(desc="The semantic keywords that are may be used in the task description")
    semantic_statements: list[dict[str, Any]] = dspy.InputField(desc="The semantic statements that are may be used in the task description")
    examples: list[dict[str, Any]] = dspy.InputField(desc="A list of examples of the inbound and outbound data")
    dsl_request: str = dspy.OutputField(desc="The DSL-compliant request statements that will be used to generate the DSPy signature")
    tools:Optional[list[str]] = dspy.OutputField(desc="A list of tools that are mentioned in the user request. If no tools are mentioned, return an empty list.")


request = "I need to have an agent analyze an email in an inbox they monitor using the 'watch_inbox' tool, and then summarize the email and send it to me in slack using the 'send_message' tool. It its part of an email chain then the agent should also create a summary of the prior converstation as part of the summary, and send it to me as well.."
dsl_composer = dspy.ChainOfThought(DSLComposer)
dsl_request = dsl_composer(user_request=request, semantic_keywords=keywords_formatted, semantic_statements=statements_formatted, examples=demos)
print(dsl_request)

In [ ]:
generate_sig = PromptGenerator()
sig = generate_sig(task=dsl_request.dsl_request, tools=dsl_request.tools if dsl_request.tools else None)
print(sig.generated_signature)


In [ ]:
import remodl
from remodl.agents import Agent, Team, Workflow, State
from remodl.agents.role import Role, expects, returns, capabilities
from remodl.agents.a2a import A2A
from remodl.agents.skills import Skill, Task, Rule,  Learn
from remodl.workflows import Step, Workflow, Decision, Exception
from remodl.helpers import aiFunction, _aiFunction
from typing import List, Dict, Any, Optional
from remodl.agents.tools import Tool, ToolResult, ToolError, MCPTools
from remodl.lexiq import Lexiq, Memory, Knowledge
from remodl.agents.profiles import Profile, with_profile, _with_profile


"""
We stay relatively close to the litellm interface for configure, as its simple.  We have certain defaults that we use.

remodl.configure -> litellm.configure. By default we have certain integrations that we use.
For models - we alias the remodl/remodl-chat-small to something effecient, like a GPT-OSS-20B, but it doesn't matter 

"""
remodl_chat = remodl.configure(model="remodl/remodl-chat-small", api_key="remodl-api-key", temperature=0.7, max_tokens=4096)
# We also allow for other models to be used, if available:
remodl_custom = remodl.configure(model="openai/gpt-4o-mini", api_key="remodl-api-key", temperature=0.7, max_tokens=4096)

# Additionally, we enable the use of "Profiles" - which are a way to configure the model for a specific task.
# We have certain defaults that we use, but we also allow for other models to be used, if available:
chat_profile = Profile(
    name="Chat",
    model="remodl/remodl-chat-small",
    temperature=0.7,
    max_tokens=4096,
    use_when = "When you need to chat with a user"
)

manager_profile = Profile(
    name="Manager",
    model="remodl/remodl-chat-large",
    temperature=0.7,
    max_tokens=4096,
    use_when = "When you need to manage a team"
)

# Profiles allow for easy switching if someone doesn't want to juggle multiple defined variable, 
#usage leverages a sentinel '@with_profile'

@with_profile(chat_profile) # or @with_profile("Chat")

def do_something():
    return "Hello, world!"


"""
We work on the following conceptual truths:
1. Atomic use of AI and LLMS as functional code enables flexibility
2. AI functions execute.
3. Agents are higher level and complex. They are variants of a ReAct agentic approach
4. Agents can work with other agents (leveraging a2a)
5. Specialized agents can create new agents
6. An end user may experience what feels like a single agent, but in fact it is a team of agents working together
7. All Remodl SDK code should be composable, declarative, and able to be "assembled" (like lego blocks)
8. If an agent or AI function can't be comprehended in natural language, then it isn't a single agent.
9. We assign special meaning to specific words:
    - Will: defines a required parameter or action
    - Should: defines a suggested parameter or action
    - May: defines an optional parameter or action
    - Can (alt "Is able to"): defines a capability
    - Must (alt "Is required", "needs to", "needs"): defines a constraint
    - Is: defines a state
    - Will: defines a required action
    - Has: defines a property
    - Uses: defines a tool


"""



t

## Cloudpickle and Remodl.save


In [ ]:
import cloudpickle

# This creates bytes in memory, no file
pickled_bytes = cloudpickle.dumps(DSLComposer)
# pickled_bytes is now a bytes object like: b'\x80\x04\x95...'

# To see a file, you need to explicitly write it:
with open('foo.pkl', 'wb') as f:
    cloudpickle.dump(DSLComposer, f)
# NOW you'll see foo.pkl created

## mcp2py test


In [ ]:
%pip install -U mcp2py fastmcp

In [ ]:
from mcp2py import load

api = load('https://api.example.com/mcp', headers={"Authorization": "Bearer YOUR_TOKEN"})


